




<a id="prereqs"></a>
# 1. Loading prereqs

In [ ]:
!pip install facenet-pytorch  # fornisce modelli pre-addestrati PyTorch per compiti di riconoscimento facciale
!pip install Pillow # aggiunge il supporto per l'apertura, la manipolazione e il salvataggio di molti diversi formati di file immagine.




<a id="load_model"></a>
# 2. Evaluating the classifier

## Load pre-trained model

In [ ]:
# utilizzo la libreria facenet_pytorch per caricare il modello InceptionResnetV1 preaddestrato sul dataset VGGFace2 e abilitare la classificazione.
from facenet_pytorch import InceptionResnetV1

resnet = InceptionResnetV1(pretrained='vggface2').eval()
resnet.classify = True  # classify indica se il modello deve emettere le probabilità di classificazione o feature embeddings. Questo perché tipicamente le reti di face recognition vengono utilizzate avvalendosi
                        # degli embeddings per poter ricreare un proprio sistema di face recognition (gallery personale)

## Define Utility Functions

In [ ]:
from io import BytesIO
from PIL import Image
import requests
from torchvision import transforms
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

def load_image(filename):
    """ carica un'immagine da un URL utilizzando la libreria requests,
    quindi la converte in un oggetto BytesIO e la apre come un'immagine utilizzando Image.open dal modulo Pillow.
    Successivamente, ridimensiona l'immagine a dimensioni 160x160 pixel e la converte in un tensore utilizzando
    transforms.ToTensor() dal modulo torchvision.transforms.
    Infine, restituisce sia il tensore dell'immagine che l'immagine aperta.
    """
    response = requests.get(filename)
    img_bytes = BytesIO(response.content)
    rsz = Image.open(img_bytes).resize((160, 160))
    tns = transforms.ToTensor()(rsz)
    return tns, rsz

## Load the labels the model was trained on

In [ ]:
# Il modello è addestrato sulle seguenti Labels:
# Carico le labels del dataset VGGFACE
fpath = tf.keras.utils.get_file('rcmalli_vggface_labels_v2.npy',
                             "https://github.com/rcmalli/keras-vggface/releases/download/v2.0/rcmalli_vggface_labels_v2.npy",
                             cache_subdir="./")
LABELS = np.load(fpath) # List of name
for i in range(len(LABELS)):
  LABELS[i] = LABELS[i].strip().replace(' ', '').replace('"', '')

labels = {}
for i in range(len(LABELS)):
  labels[LABELS[i]] = i

In [ ]:
print(LABELS)

In [ ]:
print(labels)

## Load VGGFace2 Dataset (fornito da Greco come tar.gz)

In [ ]:
import os
import random
import tarfile
import pandas as pd
import gdown

# URL del file vggface2_train.tar.gz
url = "https://drive.google.com/uc?export=download&id=1K56kVYHHDfLA2Anm7ga0tQolMwIPk6R8"
file_name = "vggface2_train.tar.gz"

# Cartella di destinazione per il download
download_folder = "./downloads"

# Se la cartella di download non esiste, creala
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Percorso completo del file scaricato
file_path = os.path.join(download_folder, file_name)

# Scarica il file se non è già presente nella cartella di download
if not os.path.exists(file_path):
    print(f"Avvio del download di {file_name}")
    gdown.download(url, file_path, quiet=False)
    print("Download completato.")
else:
    print(f"Il file {file_name} è già presente.")


## Dataset Cleaning

A partire dal file identity_meta.csv si noti come questo risulti essere formattato in modo inappropriato: l'attributo "name" contiene alcuni valori con la virgola (,) rompendo pertanto la struttura del file csv. <br> Individuata la criticità, la si risolve. <br> Inoltre, si noti come il numero di label su cui è addestrato il modello è diverso dal numero di label in meta_identity.csv. Si rimuovono quindi le labels su cui il modello non è addestrato. (Filter by name)

In [ ]:
import gdown
import pandas as pd

# URL del file identity_meta.csv
url = "https://drive.google.com/uc?export=download&id=1SXhc8m5PHxyM4lEWVEccSufIa8OiOjGW"

# Percorso di destinazione per il download del file
download_folder = "./downloads"

# Se la cartella di download non esiste, creala
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

# Percorso completo del file scaricato
file_path = os.path.join(download_folder, "identity_meta.csv")

# Scarica il file se non è già presente nella cartella di download
if not os.path.exists(file_path):
    print("Avvio del download di identity_meta.csv")
    gdown.download(url, file_path, quiet=False)
    print("Download completato.\n")
else:
    print("Il file identity_meta.csv è già presente.")

print("File path: ", file_path)

import csv
import csv
import pandas as pd

# Lista per memorizzare le righe con numero errato di colonne
invalid_rows = []

# Lista per memorizzare le righe corrette
valid_rows = []

print('\n\nCleaning rows')
# Apri il file CSV e leggi le righe ignorando quelle con un numero errato di colonne
with open(file_path, 'r', newline='', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        # Verifica se il numero di colonne è corretto
        if len(row) == 5:
            row[1] = row[1].strip().replace(' ', '').replace('"', '')
            valid_rows.append(row)
        else:
            invalid_rows.append(row)
            print(f"Riga {csv_reader.line_num} -> {row}")
            name = row[1].strip('"').strip().replace(',', '') + " " + row[2].strip('"').strip()
            row[1] = name.strip().replace(' ', '').replace('"', '')
            del row[2]
            print(f"Riga aggiustata: {row}")
            valid_rows.append(row)

identity_meta_clean = pd.DataFrame(valid_rows[1:], columns=valid_rows[0])

identity_meta_clean.to_csv("meta_identity_clean.csv", index=False)

# filtro sulle label su cui ha appreso il modello
for row in valid_rows[1:]:
  if row[1] in labels:
    pass
  else:
     valid_rows.remove(row)

# Costruisci un DataFrame da data
identity_meta = pd.DataFrame(valid_rows[1:], columns=valid_rows[0])

print("\n\nidentity meta data frame:\n", identity_meta)
identity_meta.to_csv("meta_identity_NN1.csv", index=False)
# create a mapping from name and id --> il modello dà un id che è associato ad un name che nel
# nostro dataset risponde ad un id diverso... quindi ci serve il mapping tra name and actual id
name_to_id = {}
for index, row in identity_meta.iterrows():
    # Ora puoi accedere ai valori di ogni riga come segue:
    class_id = row['Class_ID']
    name = row['Name']
    name_to_id[name]=class_id

In [ ]:
print(name_to_id)

## Preparing test set
Seleziona 100 identità a caso come da richiesta, inizializzando un seed per la riproducibilità dell'esperimento

In [ ]:
import pandas as pd
import random
seed = 42

# Estrai 100 righe casuali dal DataFrame
selected_df = identity_meta.sample(n=100, random_state=seed)

# Salva le righe selezionate in un nuovo file CSV
selected_csv = "selected_data.csv"
selected_df.to_csv(selected_csv, index=False)

print("100 righe casuali sono state estratte e salvate in:", selected_csv)

In [ ]:
df = pd.read_csv(selected_csv)
print(df)

In [ ]:
# esegui questo se vuoi lavorare con il dataset estratto
!tar -xzf "./downloads/vggface2_train.tar.gz"

train/n001356/0162_01.jpg: Can't create '\\?\g:\Drive condivisi\AI4CYBSEC\train\n001356\0162_01.jpg': File exists
train/n002014/0093_01.jpg: Can't create '\\?\g:\Drive condivisi\AI4CYBSEC\train\n002014\0093_01.jpg': No such file or directory
train/n002014/0771_01.jpg: Can't create '\\?\g:\Drive condivisi\AI4CYBSEC\train\n002014\0771_01.jpg': No such file or directory
train/n002014/0045_06.jpg: Can't create '\\?\g:\Drive condivisi\AI4CYBSEC\train\n002014\0045_06.jpg': No such file or directory
train/n002014/0044_01.jpg: Can't create '\\?\g:\Drive condivisi\AI4CYBSEC\train\n002014\0044_01.jpg': No such file or directory
tar: Truncated input file (needed 50688 bytes, only 0 available)
tar: Error exit delayed from previous errors.


Aternativa alla cella precedente

In [ ]:
import os
import shutil
import random
from tqdm import tqdm

seed = 42

def extract_images(root, classID, num_imgs, input_folder, output_folder, seed):
    random.seed(seed)
    # Percorso della cartella contenente le immagini estratte
    input_class_folder = os.path.join(input_folder, str(classID))
    # Percorso della cartella di output per questa classe
    output_class_folder = os.path.join(output_folder, str(classID))

    # Crea la cartella di output per questa classe se non esiste già
    os.makedirs(output_class_folder, exist_ok=True)

    # Elenco dei file immagine nella cartella della classe
    image_files = [f for f in os.listdir(input_class_folder) if os.path.isfile(os.path.join(input_class_folder, f))]

    # Seleziona un massimo di num_imgs immagini in modo casuale (se ce ne sono meno, seleziona tutte)
    selected_images = random.sample(image_files, min(num_imgs, len(image_files)))
    
    



    # Copia e salva le immagini selezionate
    for image_name in selected_images:
        input_image_path = os.path.join(input_class_folder, image_name)
        output_image_path = os.path.join(output_class_folder, image_name)
        shutil.copyfile(input_image_path, output_image_path)

root = "train"
num_imgs = 10
input_folder = "./train"
output_folder = "./face_dataset/test_set"

for classID in tqdm(df["Class_ID"], desc='Processing classes', unit='class'):
    extract_images(root, classID, num_imgs, input_folder, output_folder, seed)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!mkdir /content/drive/Shareddrives/AI4CYBSEC/face_dataset

In [ ]:
!cp -r ./test_set /content/drive/Shareddrives/AI4CYBSEC/face_dataset/test_set

In [ ]:
!cp -r ./selected_data.csv /content/drive/Shareddrives/AI4CYBSEC/face_dataset/selected_data.csv

In [ ]:
!cp -r ./meta_identity_NN1.csv /content/drive/Shareddrives/AI4CYBSEC/face_dataset/meta_identity_NN1.csv

In [ ]:
!cp -r ./meta_identity_clean.csv /content/drive/Shareddrives/AI4CYBSEC/face_dataset/meta_identity_clean.csv

In [ ]:
!cp -r ./downloads/identity_meta.csv /content/drive/Shareddrives/AI4CYBSEC/face_dataset/identity_meta.csv

### Utilities

In [ ]:
def load_image(file_path):
    """ carica un'immagine da un percorso e la apre come un'immagine utilizzando Image.open dal modulo Pillow.
    Successivamente, ridimensiona l'immagine a dimensioni 160x160 pixel e la converte in un tensore utilizzando
    transforms.ToTensor() dal modulo torchvision.transforms.
    Infine, restituisce sia il tensore dell'immagine che l'immagine aperta.
    """
    rsz = Image.open(file_path).resize((160, 160))
    tns = transforms.ToTensor()(rsz)
    return tns, rsz

def make_inference(tensor_image, name_to_id):
  """
  prende in ingresso il tensore dell'immagine e ritorna la label associata alla predizione della rete
  """
  if len(tensor_image.shape) == 3:
      tensor_image = tensor_image.unsqueeze(0)
  probs = resnet(tensor_image)
  target_class = np.array(probs[0].detach().numpy()).argmax()
  return name_to_id[LABELS[target_class]], target_class

def plot_image(original_image, original_label):
  """
  prende in ingresso le PIL.Image del campione originale e del corrispondete adversarial sample e li plotta
  """
  plt.figure()
  plt.matshow(original_image)
  plt.title("Model Prediction: {}".format(original_label))
  plt.show()

### Test model in inference on a random identity of test set

In [ ]:
import os
import random

# Percorso della directory ./test_set
test_set_path = "/content/drive/Shareddrives/AI4CYBSEC/face_dataset/test_set"

# Ottieni un elenco di tutte le cartelle in ./test_set
subdirectories = [f for f in os.listdir(test_set_path) if os.path.isdir(os.path.join(test_set_path, f))]

# Seleziona casualmente una cartella
random_folder = random.choice(subdirectories)

# Percorso della cartella selezionata casualmente
random_folder_path = os.path.join(test_set_path, random_folder)

# Itera su tutti i file nella cartella selezionata casualmente
for file_name in os.listdir(random_folder_path):
    file_path = os.path.join(random_folder_path, file_name)
    if os.path.isfile(file_path):
        tns, rsz = load_image(file_path)
        pred, target_class = make_inference(tns, name_to_id)
        plot_image(rsz, pred)

        name = LABELS[target_class]

        # double check con il file identity_meta.csv
        for idx,row in identity_meta.iterrows():
          if row['Class_ID'] == pred and row["Name"] == name:
            print('Double check passed')